# Approx Lab

The goal of this lab is to gain experience using approximation algorithms.

For the purpose of this assignment, you will be implementing the knapsack problem. Before continuing, the following cell contains any dependencies you may need for this assignment.

In [1]:
import pandas as pd

## Background Info

### Knapsack Problem
We are actually going to be looking at a specific (hopefully fun) implementation of the knapsack problem.

The traditional knapsack problem asks you to pack a set of items, with given weights and values into a "knapsack" with a maximum capacity. You cannot pack more items than the capacity of the knapsack. The goal is to select a subset of items that maximize the total value with what can fit in the knapsack. The name "knapsack" comes from the similarity to a hiker trying to pack a bag without going over a certain weight.

The knapsack problem is an NP-Hard optimization problem, which is why it is one of the classic approximation algorithms.

### Specifications for this Lab
The variation that we will be doing in this assignment is referred to as an 0/1 knapsack problem, meaning that items must be completely put in the knapsack or they must be left behind (you can't have a fractional portion of an item). We will be representing this as a simple fantasy sports league - specifically Formula 1 drivers. 

You are provided with a list of F1 drivers and constructors with given costs and points as well as a budget of $100M. You will be selecting five drivers and two constructors (teams). 

### Scoring Information
Points are gained from qualification results for drivers and constructors as well as race results for drivers and constructors. 

<!-- Qualifications
- Drivers score based on their resulting position 
- Constructor score the combined total of their two drivers in qualifying plus a modifier based on how far both their drivers get. -->
<!-- 
| Drivers    | Points |
| -------- | ------- |
| Pole Position (1st Place) | 10 |
| 2nd Place | 9 |
| 3rd Place | 8 |
| 4th Place | 7 |
| 5th Place | 6 |
| 6th Place | 5 |
| 7th Place | 4 |
| 8th Place | 3 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| No Time Set | 0 |
| Disqualification | 0 | -->


<!-- | Constructors | Points |
| ----------- | ------- |
| Neither Driver Reaches Q2 | -1 |
| One Driver Reaches Q2 | 1 |
| Both Drivers Reach Q2 | 5 |
| Both Drivers Reach Q3 | 10 | -->


<!-- Race
- Drivers score based on their resulting position, position difference from starting order, and some additional bonuses
- Constructors score a combined total of their two drivers' results (excluding the Driver of the Day bonus) ~~plus a modifier based on pitstop times~~ -->

<!-- | Drivers | Points |
| -------- | ------- |
| Positions Gained | 1/position |
| Positions Lost | -1/position |
| Overtakes Made | 1/overtake |
| Fastest Lap | 10 |
| Driver of the Day | 10 | -->

<!-- | Race Result | Points |
| -------- | ------- |
| 1st Place | 25 |
| 2nd Place | 18 |
| 3rd Place | 15 |
| 4th Place | 12 |
| 5th Place | 10 |
| 6th Place | 8 |
| 7th Place | 6 |
| 8th Place | 4 |
| 9th Place | 2 |
| 10th Place | 1 |
| 11th-20th Place | 0 |
| DNF | -20 |
| Disqualification | -20 | -->

Fortunately for you, we have access to the current point standings post-2025 China GP for drivers & constructors, which are available in `drivers.csv` & `constructors.csv`.

*Note: The costs are in millions, so 30 is actually 30 M.

### Questions (Answered in Various Parts of the Lab): 
1. What is the general setup of the problem?
2. Determining the mathematical functions of the problem
3. Implementing a greedy algorithm


## A. Assessing the Problem

The values that are provided for this lab are from the points distribution PRIOR to the Japan Grand Prix (which means we can actually compare with the race results at the submission deadline!)

Create a fantasy team https://fantasy.formula1.com/en/create-team for the upcoming Grand Prix to get a better idea of what drivers, constructors, points, and the budget are. Who is on your team & how much did you spend? Don't spend too much time on this, it really is just to get familiar with the format.

- Drivers: Ocon, Albon, Bearman, Stroll, Norris
- Constructors: Mercedes, Haas
- Spent: $98M
- Points: 412

What represents the "items" in the knapsack? 

- The selected drivers and constructors

What represents the weights?

- Their cost

What represents the values?

- Their points

In [81]:
# Load the data from the CSV files into a format that you are comfortable working with
# This can be arrays, dictionaries, or pd DataFrames
drivers = pd.read_csv('drivers.csv')
constructors = pd.read_csv('constructors.csv')
# Print/Display your data as well to confirm that you have successfully saved it.
print(drivers)
print(constructors)

                   Driver  Points  Cost
0            Lando Norris     100  29.6
1   Andrea Kimi Antonelli      61  19.3
2          George Russell      60  21.4
3          Max Verstappen      59  28.6
4           Oscar Piastri      55  23.0
5            Lance Stroll      33   9.3
6            Esteban Ocon      32   8.1
7          Alexaner Albon      28  12.8
8         Nico Hulkenberg      22   7.6
9          Oliver Bearman      22   6.7
10         Lewis Hamilton       3  23.6
11           Yuki Tsunoda       0  16.8
12            Liam Lawson       0   8.4
13        Charles LeClerc       0  25.3
14           Pierre Gasly      -7  10.6
15           Isack Hadjar      -9   5.0
16            Jack Doohan      -9   6.0
17           Carlos Sainz     -11  11.9
18      Gabriel Bortoleto     -11   4.8
19        Fernando Alonso     -36   7.6
       Constructor  Points  Cost
0          McLaren     172  30.6
1         Mercedes     136  23.3
2  Red Bull Racing      86  25.4
3             Haas      61  

# B. Integer Program

Before implementing our approx algorithm, we need to first consider the integer program. 

Given the following variables, create an integer program that maximizes the total points of drivers while remaining under the budget: 
- $x_i$ is the decision variable that determines whether driver $i$ is included in the team or not
- $p_i$ is the # of points that driver $i$ is worth
- $c_i$ is the cost of driver $i$

For now, we will focus on just the drivers and ignore the constructors. We will also temporarily reduce our budget to **$50M**

While we are not planning on implementing the full integer program, it helps us quantify parts of the problem


$\text{maximize} \quad \sum_{i=0}^{I} x_i p_i$  \
subject to:
- $\sum_{i=0}^{I} x_i c_i \leq 50$
- $\sum_{i=0}^{I} x_i \leq 5$ 

where $I$ is the total number of drivers


## Decision Variable & Constraints

We can make helper functions for our future algorithm based on constraints that we derive when forming the IP: 
1. Which drivers should be included in our solution? 
2. What is our maximum cost?
3. What is our maximum # of drivers? 

In [ ]:
BUDGET = 50 # in millions
MAX_DRIVERS = 5

def remaining_budget(selected_drivers: pd.DataFrame) -> float:
    return BUDGET - selected_drivers["Cost"].sum()

# C. Greedy Algorithm


There are multiple ways we can go about solving the knapsack problem. For this lab, we will be focusing on a greedy algorithm and a variant of that.

Let's start with a basic greedy algorithm:
1. Calculate the value-to-cost ratio for each driver 
2. Sort drivers in non-increasing order of value-to-cost
2. Greedily select drivers from list until either budget or max drivers constraint is hit

Use the helper methods from the previous part in your implementation.

In [ ]:
def value_to_cost(items: pd.DataFrame) -> pd.DataFrame:
    out = items.copy()
    out["UnitCost"] = items["Points"] / items["Cost"]
    return out



def sort_items(items: pd.DataFrame) -> pd.DataFrame:
    return items.sort_values(by="UnitCost", ascending=False)


def greedy_driver_select(drivers):
    drivers = sort_items(value_to_cost(drivers))
    selected_drivers = pd.DataFrame(columns=drivers.columns)
    for _, row in drivers.iterrows():
        if (
            remaining_budget(selected_drivers) > row["Cost"]
            and len(selected_drivers) < MAX_DRIVERS
        ):
            # Keep adding drivers if enough budget and below max drivers
            selected_drivers.loc[len(selected_drivers)] = row
    return selected_drivers

selected = greedy_driver_select(drivers)
print(selected)
print(f"Total points: {sum(selected["Points"])}, Total cost: {sum(selected["Cost"])}")

         Driver Points  Cost  UnitCost
0  Esteban Ocon     32   8.1  3.950617
1  Lance Stroll     33   9.3  3.548387
2  Lando Norris    100  29.6  3.378378
Total points: 165, Total cost: 47.0


Try running your basic greedy algorithm. 

You may notice a few flaws:
1. The greedy algorithm is only looking at value-to-cost ratio, which may miss out on an overall improvement in value that still fits within the budget
2. Depending on your implementation, you may not currently be meeting the "5 total drivers" constraint - this is fine


## Redux Greedy Algorithm
We are going to next implement a small adjustment to this algorithm in order to improve its performance. \
This is a 2-approximation for the knapsack problem. 

1. Calculate the value-to-cost ratio for each driver
2. Sort drivers in non-increasing order of value-to-cost
3. Greedily add items until we hit an item that is too big
4. Compare the too big item against the already existing list of items & pick the better of the two

It might help to make a separate function to handle #4. 

In [112]:
def replace_if_optimal(selected_drivers: pd.DataFrame, new_driver: pd.Series) -> pd.DataFrame:
    for idx, row in selected_drivers.iterrows():
        if (
            # If enough budget
            new_driver["Cost"] - row["Cost"] <= remaining_budget(selected_drivers)
            # If driver is more points
            and new_driver["Points"] > row["Points"]
        ):
            # Replace the driver
            selected_drivers.loc[idx] = new_driver
            return selected_drivers
    # Otherwise return original df
    return selected_drivers


def redux_greedy_driver_select(drivers):
    drivers = sort_items(value_to_cost(drivers))
    selected_drivers = pd.DataFrame(columns=drivers.columns)
    for _, row in drivers.iterrows():
        if (
            remaining_budget(selected_drivers) >= row["Cost"]
            and len(selected_drivers) < MAX_DRIVERS
        ):
            # Neither constraint reached
            selected_drivers.loc[len(selected_drivers)] = row
        else:
            selected_drivers = replace_if_optimal(selected_drivers, row)

    return selected_drivers


# This algorithm doesn't actually perform differently so I added an example new driver to test

# test = drivers.copy()
# test.loc[len(test)] = ["Al Sacco", 34, 12.3]
# selected = redux_greedy_driver_select(test)

selected = redux_greedy_driver_select(drivers)
print(selected)
print(f"Total points: {sum(selected["Points"])}, Total cost: {sum(selected["Cost"])}")

         Driver Points  Cost  UnitCost
0  Esteban Ocon     32   8.1  3.950617
1  Lance Stroll     33   9.3  3.548387
2  Lando Norris    100  29.6  3.378378
Total points: 165, Total cost: 47.0


# D. Reintroducing Constructors

We've been neglecting our constructors so far and cutting our budget. Let's make them feel included in our processes again. 

## Integer Program

We should start by redefining our original IP to see what constraints we need to add. \
We've added some more variables to our IP: 
- $x_j$ is the decision variable that determines whether constructor $j$ is included in the team or not 
- $p_j$ is the # of points that constructor $j$ is worth
- $c_j$ is the cost of constructor $j$

Our budget has also been increased back to **100M**. 


$\text{maximize} \quad \sum_{i=0}^{I} x_i p_i + \sum_{j=0}^{J} x_j p_j$  \
subject to:
- $\sum_{i=0}^{I} x_i c_i + \sum_{j=0}^{J} x_j c_j \leq 100$
- $\sum_{i=0}^{I} x_i \leq 5$ 
- $\sum_{j=0}^{J} x_j \leq 2$

where $I$ is the total number of drivers and $J$ is the total number of constructors


## Constraints 

To redefine our greedy algorithm to include constructors, we have to also write helpers that correspond to our new constraints.

In [114]:
BUDGET = 100  # in millions
MAX_DRIVERS = 5
MAX_CONSTRUCTORS = 2

def remaining_budget(selected: pd.DataFrame) -> float:
    return BUDGET - selected["Cost"].sum()

def is_selection_full(selected: pd.DataFrame, current: pd.Series) -> bool:
    """
    Returns False if less than the max number of drivers or constructors,
    depending on the type of `current` are selected. Returns True otherwise.
    """
    if (
        current["Type"] == "Driver"
        and len(selected[selected["Type"] == "Driver"]) < MAX_DRIVERS
    ):
        return False
    if (
        current["Type"] == "Constructor"
        and len(selected[selected["Type"] == "Constructor"])  < MAX_CONSTRUCTORS
    ):
        return False
    return True

## Greedy Algorithm

Update the greedy algorithm to calculate for buying both constructors and drivers. \
Remember, your list of non-increasing value-to-cost ratios should be a combination of both drivers & constructors. 

In [124]:
def greedy_f1(drivers, constructors, redux=True):
    drivers = drivers.copy()
    constructors = constructors.copy()
    # Add type label
    drivers["Type"] = "Driver"
    constructors["Type"] = "Constructor"
    # Rename columns to "Name"
    drivers.rename(columns={"Driver": "Name"}, inplace=True)
    constructors.rename(columns={"Constructor": "Name"}, inplace=True)
    # Combine to single df
    combined = sort_items(value_to_cost(pd.concat([drivers, constructors])))

    selected = pd.DataFrame(columns=combined.columns)

    for _, row in combined.iterrows():
        if (
            remaining_budget(selected) > row["Cost"]
            and not is_selection_full(selected, row)
        ):
            # Keep adding drivers/constructors if enough budget and below max drivers/constructors
            selected.loc[len(selected)] = row
        elif redux:
            # Replace if optimal
            selected = replace_if_optimal(selected, row)
    return selected

selected = greedy_f1(drivers, constructors, redux=False)
print("==================== Not Redux ====================")
print(selected)
print(f"Total points: {sum(selected["Points"])}, Total cost: {sum(selected["Cost"])}")

print("")

selected = greedy_f1(drivers, constructors)
print("==================== Redux ====================")
print(selected)
print(f"Total points: {sum(selected["Points"])}, Total cost: {sum(selected["Cost"])}")

==================== Not Redux ====================
              Name Points  Cost         Type  UnitCost
0             Haas     61   8.2  Constructor  7.439024
1         Mercedes    136  23.3  Constructor   5.83691
2     Esteban Ocon     32   8.1       Driver  3.950617
3     Lance Stroll     33   9.3       Driver  3.548387
4     Lando Norris    100  29.6       Driver  3.378378
5   Oliver Bearman     22   6.7       Driver  3.283582
6  Nico Hulkenberg     22   7.6       Driver  2.894737
Total points: 406, Total cost: 92.8

==================== Redux ====================
             Name Points  Cost         Type  UnitCost
0         McLaren    172  30.6  Constructor  5.620915
1        Mercedes    136  23.3  Constructor   5.83691
2    Lando Norris    100  29.6       Driver  3.378378
3    Lance Stroll     33   9.3       Driver  3.548387
4  Oliver Bearman     22   6.7       Driver  3.283582
Total points: 463, Total cost: 99.5
